In [113]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from github import Github
from datetime import timedelta

In [114]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [115]:
# Calculate number of days to include in plots
n_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

In [116]:
# Create the renderer we want
alt.renderers.enable(renderer);

In [117]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = """
{
  search(%s) {
    issueCount
    pageInfo {
        endCursor
        hasNextPage
    }
    nodes {
      ... on PullRequest {
        state
        id
        title
        url
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        mergedBy {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              authorAssociation
              createdAt
              lastEditedAt
              url
              repository {
                id
              }
              author {
                login
                url
              }
            }
          }
        }
      }
      ... on Issue {
        url
        title
        state
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              author {
                login
                url
              }
              authorAssociation
              createdAt
              lastEditedAt
              repository {
                id
              }
              url
            }
          }
        }
      }
    }
  }
}
"""
    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template % ', '.join(search_query)
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['username'] = self.data['author'].map(lambda a: a['login'])
        self.data['username_url'] = self.data['author'].map(lambda a: a['url'])
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [118]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [119]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [120]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [121]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [122]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (101 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (53 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (15 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (16 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [123]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (101 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (53 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Rename this category to Jupyter Server?](https://discourse.jupyter.org/t/rename-this-category-to-jupyter-server) (21 posts)
* [Creating a specification for reproducible repositories](https://discourse.jupyter.org/t/creating-a-specification-for-reproducible-repositories) (21 posts)

In [124]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  10       accessibility           2      Special Topics   
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  6              discuss           2          JupyterHub   
  2           JupyterHub           1          JupyterHub   
  8   Enterprise Gateway           2  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  
                            parent/category  
  10           Special Topics/accessibility  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [125]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [What is with the weird jovyan user?](https://discourse.jupyter.org/t/what-is-with-the-weird-jovyan-user) (12 posts)
* [Extension development very slow](https://discourse.jupyter.org/t/extension-development-very-slow) (5 posts)
* [What happens when the Hub restarts?](https://discourse.jupyter.org/t/what-happens-when-the-hub-restarts) (5 posts)
* [Technology choices for new Jupyter.org website](https://discourse.jupyter.org/t/technology-choices-for-new-jupyter-org-website) (4 posts)
* [Resources for giving talks about the JupyterHub and Binder projects](https://discourse.jupyter.org/t/resources-for-giving-talks-about-the-jupyterhub-and-binder-projects) (4 posts)
* [MP Workshop on MyBinder](https://discourse.jupyter.org/t/mp-workshop-on-mybinder) (4 posts)
* [Ipywidgets for jupyterlab 1.0](https://discourse.jupyter.org/t/ipywidgets-for-jupyterlab-1-0) (3 posts)
* [What is the best way to add numbers to equations in a Markdown cell in a Jupyter Lab notebook?](https://discourse.jupyter.org/t/what-is-the-best-way-to-add-numbers-to-equations-in-a-markdown-cell-in-a-jupyter-lab-notebook) (3 posts)
* [How do I enable a bash kernel?](https://discourse.jupyter.org/t/how-do-i-enable-a-bash-kernel) (3 posts)
* [How to add a new route](https://discourse.jupyter.org/t/how-to-add-a-new-route) (2 posts)

In [126]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked parent_category  \
  8                                Q&A           2             Q&A   
  11                          Notebook           2        Notebook   
  2                               Meta           1            Meta   
  7                         JupyterLab           4      JupyterLab   
  10                              team           2      JupyterHub   
  0   Zero to JupyterHub on Kubernetes           1      JupyterHub   
  3                         JupyterHub           1      JupyterHub   
  5            The Littlest JupyterHub           0      JupyterHub   
  4                     Community Chat           2  Community Chat   
  9                   mybinder.org ops           1          Binder   
  1                          repo help           0          Binder   
  6                            discuss           0          Binder   
  
                                  parent/category  
  8                                       Q&A/Q&A  
  11                            Notebook/Notebook  
  2                                     Meta/Meta  
  7                         JupyterLab/JupyterLab  
  10                              JupyterHub/team  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  3                         JupyterHub/JupyterHub  
  5            JupyterHub/The Littlest JupyterHub  
  4                 Community Chat/Community Chat  
  9                       Binder/mybinder.org ops  
  1                              Binder/repo help  
  6                                Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'Meta', 'JupyterLab', 'team', 'Zero to JupyterHub on Kubernetes', 'JupyterHub', 'The Littlest JupyterHub', 'Community Chat', 'mybinder.org ops', 'repo help', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in Discourse

These are the top posters and top "like-ers" in the Discourse

In [127]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [128]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [129]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             77    6          224             126         162        1433   
  1             90    3          203             118         136        1008   
  2             66   24           17              57          76         696   
  3             85    8           10              61          56         847   
  4             65   12            9              46          47         858   
  5             73  116           23              54          47         653   
  6             77   73           18              31          34        1592   
  7             14  220            1              24          27         188   
  8             81   30           77              19          24         330   
  9             61  239            4               7          23         429   
  10            32  485           43              20          21         260   
  11            55   99           38              16          21         508   
  12            18  234            5               3          16          84   
  13            33  217           17               6          14         180   
  14            32  419            0               3          14          78   
  15            33  372            6              13          14         210   
  16            86   77            6              11          14         816   
  17            64   11           59               9          12         854   
  18            28   60           11               8          12         334   
  19            13  390            2               1          11          79   
  20            36  432           28               6          10         272   
  21            20   34            2              16           9         148   
  22            45  222            3              14           9         143   
  23             9  679            1              10           9          85   
  25             9  254           11               1           8          92   
  24            10  510            0               6           8          10   
  27            17  462            0               1           7          73   
  28            14    9            0               9           7          93   
  29            16  227           10               2           7         235   
  30            11   71            0               4           7          76   
  31            10  566            0               1           7          32   
  32             6  686            0               2           7          91   
  33            22  333           11               3           7          43   
  26            14  487            0               0           7          83   
  39            21  281            3               8           6          55   
  42            26  134            5              16           6         277   
  41            49  303            5               3           6         216   
  40            51  298            0               4           6         345   
  37            13  145            0               7           6          50   
  38             6  565            3               4           6          35   
  36            18    4            0              17           6          44   
  35            18  237            1              25           6          51   
  34             9  195            0               1           6          33   
  43            11  247            6              14           5         183   
  44            11  430            2               3           5          41   
  45             6  556            3               6           5          65   
  46             6  640            0               0           5          22   
  47             8  562            0               0           5          25   
  48            33  184           14               2           5  

**Discourse users sorted by likes given**

In [130]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             77    6          224             126         162        1433   
  1             90    3          203             118         136        1008   
  8             81   30           77              19          24         330   
  17            64   11           59               9          12         854   
  10            32  485           43              20          21         260   
  11            55   99           38              16          21         508   
  20            36  432           28               6          10         272   
  5             73  116           23              54          47         653   
  6             77   73           18              31          34        1592   
  2             66   24           17              57          76         696   
  13            33  217           17               6          14         180   
  48            33  184           14               2           5         280   
  33            22  333           11               3           7          43   
  18            28   60           11               8          12         334   
  25             9  254           11               1           8          92   
  29            16  227           10               2           7         235   
  3             85    8           10              61          56         847   
  4             65   12            9              46          47         858   
  15            33  372            6              13          14         210   
  43            11  247            6              14           5         183   
  16            86   77            6              11          14         816   
  49            68  229            6               0           5         141   
  12            18  234            5               3          16          84   
  41            49  303            5               3           6         216   
  42            26  134            5              16           6         277   
  9             61  239            4               7          23         429   
  45             6  556            3               6           5          65   
  38             6  565            3               4           6          35   
  22            45  222            3              14           9         143   
  39            21  281            3               8           6          55   
  21            20   34            2              16           9         148   
  44            11  430            2               3           5          41   
  19            13  390            2               1          11          79   
  35            18  237            1              25           6          51   
  23             9  679            1              10           9          85   
  7             14  220            1              24          27         188   
  40            51  298            0               4           6         345   
  46             6  640            0               0           5          22   
  47             8  562            0               0           5          25   
  14            32  419            0               3          14          78   
  36            18    4            0              17           6          44   
  37            13  145            0               7           6          50   
  34             9  195            0               1           6          33   
  31            10  566            0               1           7          32   
  30            11   71            0               4           7          76   
  28            14    9            0               9           7          93   
  27            17  462            0               1           7          73   
  26            14  487            0               0           7          83   
  24            10  510            0               6           8  

**Discourse users sorted by likes received**

In [131]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             77    6          224             126         162        1433   
  1             90    3          203             118         136        1008   
  3             85    8           10              61          56         847   
  2             66   24           17              57          76         696   
  5             73  116           23              54          47         653   
  4             65   12            9              46          47         858   
  6             77   73           18              31          34        1592   
  35            18  237            1              25           6          51   
  7             14  220            1              24          27         188   
  10            32  485           43              20          21         260   
  8             81   30           77              19          24         330   
  36            18    4            0              17           6          44   
  11            55   99           38              16          21         508   
  42            26  134            5              16           6         277   
  21            20   34            2              16           9         148   
  22            45  222            3              14           9         143   
  43            11  247            6              14           5         183   
  15            33  372            6              13          14         210   
  16            86   77            6              11          14         816   
  23             9  679            1              10           9          85   
  28            14    9            0               9           7          93   
  17            64   11           59               9          12         854   
  39            21  281            3               8           6          55   
  18            28   60           11               8          12         334   
  9             61  239            4               7          23         429   
  37            13  145            0               7           6          50   
  20            36  432           28               6          10         272   
  45             6  556            3               6           5          65   
  24            10  510            0               6           8          10   
  13            33  217           17               6          14         180   
  30            11   71            0               4           7          76   
  38             6  565            3               4           6          35   
  40            51  298            0               4           6         345   
  44            11  430            2               3           5          41   
  33            22  333           11               3           7          43   
  41            49  303            5               3           6         216   
  14            32  419            0               3          14          78   
  12            18  234            5               3          16          84   
  29            16  227           10               2           7         235   
  32             6  686            0               2           7          91   
  48            33  184           14               2           5         280   
  25             9  254           11               1           8          92   
  34             9  195            0               1           6          33   
  31            10  566            0               1           7          32   
  27            17  462            0               1           7          73   
  19            13  390            2               1          11          79   
  26            14  487            0               0           7          83   
  46             6  640            0               0           5          22   
  47             8  562            0               0           5  

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [132]:
# Define colors we'll use for GitHub membership
import seaborn as sns
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_colors = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_colors]

In [133]:
from datetime import timedelta

orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/nteract](https://github.com/nteract)

In [134]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-20** to **2019-07-19**

In [135]:
responses = []
for org in github_orgs:
    query_prs = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
merged = pd.concat([ii.data for ii in responses])
merged = merged.query('username not in @bot_names')

Found 249 items, which will take 5 pages
Found 263 items, which will take 6 pages
Found 235 items, which will take 5 pages
Found 95 items, which will take 2 pages
Found 54 items, which will take 2 pages
Found 3 items, which will take 1 pages
Found 152 items, which will take 4 pages


In [136]:
# Add a PR-specific field for closed PRs
merged['closed_by'] = merged['mergedBy'].map(lambda a: a['login'])
merged['closed_by_url'] = merged['mergedBy'].map(lambda a: a['url'])

In [137]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [138]:
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                           repo  author
  80          nteract                        nteract      40
  77          nteract                       hydrogen      39
  75          nteract                      bookstore      36
  82          nteract                      papermill      16
  81          nteract                     nteract.io       9
  76          nteract                       commuter       3
  83          nteract                      scrapbook       3
  79          nteract                 minimal-plotly       2
  85          nteract                  vega-embed-v3       2
  78          nteract                   kernel-relay       1
  84          nteract                           vdom       1
  60       jupyterlab                     jupyterlab     165
  64       jupyterlab                 jupyterlab-git      13
  62       jupyterlab       jupyterlab-data-explorer      10
  71       jupyterlab              jupyterlab_server       9
  63       jupyterlab                jupyterlab-demo       8
  59       jupyterlab              jupyter-renderers       7
  61       jupyterlab          jupyterlab-commenting       5
  74       jupyterlab             theme-cookiecutter       3
  58       jupyterlab      extension-cookiecutter-ts       2
  65       jupyterlab              jupyterlab-github       2
  69       jupyterlab                 jupyterlab-toc       2
  73       jupyterlab  scipy2019-jupyterlab-tutorial       2
  56       jupyterlab                       debugger       1
  57       jupyterlab      extension-cookiecutter-js       1
  66       jupyterlab               jupyterlab-latex       1
  67       jupyterlab    jupyterlab-metadata-service       1
  68       jupyterlab           jupyterlab-telemetry       1
  70       jupyterlab            jupyterlab_pygments       1
  72       jupyterlab     mimerender-cookiecutter-ts       1
  ..              ...                            ...     ...
  30  jupyter-widgets                   midicontrols       1
  32  jupyter-widgets                     traittypes       1
  34  jupyter-widgets            widget-cookiecutter       1
  22          jupyter                       notebook      48
  20          jupyter                       nbgrader      43
  24          jupyter                    repo2docker      30
  8           jupyter             enterprise_gateway      25
  18          jupyter                      nbconvert      16
  26          jupyter                      telemetry      15
  14          jupyter                 jupyter_client      12
  12          jupyter                 jupyter-sphinx      11
  23          jupyter                      qtconsole      11
  11          jupyter                   jupyter-book      10
  13          jupyter              jupyter.github.io       8
  16          jupyter                 jupyter_server       8
  7           jupyter                  docker-stacks       7
  15          jupyter                   jupyter_core       7
  21          jupyter                       nbviewer       4
  10          jupyter                        jupyter       3
  17          jupyter                          nb2kg       2
  9           jupyter                     governance       1
  19          jupyter                         nbdime       1
  25          jupyter                        roadmap       1
  5           ipython                        ipython      43
  6           ipython                      traitlets       5
  3           ipython                      ipykernel       4
  4           ipython                    ipyparallel       2
  0   binder-examples                     data-quilt       1
  1   binder-examples             getting-data-xroot       1
  2   binder-examples                          latex       1
  
  [86 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nteract', 'hydrogen', 'bookstor

### A list of merged PRs by project

In [139]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [140]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['username']
            user_url = pr['username_url']
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']
            pr_closedby_url = pr['closed_by_url']
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [141]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'username', 'authorAssociation']).count()['author'].reset_index().rename(columns={'author': "authored"})
closed_by_repo = merged.groupby(['org', 'repo', 'closed_by']).count()['author'].reset_index().rename(columns={'author': "closed", "closed_by": "username"})

merged_stats = pd.merge(closed_by_repo, merged_by_repo, on=['org', 'repo', 'username'], how='outer')
merged_stats = pd.melt(merged_stats, id_vars=['org', 'repo', 'username'], var_name="kind", value_name="count")
merged_stats = merged_stats.replace(np.nan, 0)

In [142]:
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org                repo         username authorAssociation  \
  0  binder-examples          data-quilt      matheusmota              NONE   
  1  binder-examples  getting-data-xroot  matthewfeickert       CONTRIBUTOR   
  2  binder-examples               latex        yuvipanda            MEMBER   
  
     authored  
  0         1  
  1         1  
  2         1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'authored',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'PR authors for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org         repo        username authorAssociation  authored
  3   ipython    ipykernel         Chilipp       CONTRIBUTOR         1
  4   ipython    ipykernel         hroncok       CONTRIBUTOR         1
  5   ipython    ipykernel           minrk            MEMBER         1
  6   ipython    ipykernel        steff456              NONE         1
  7   ipython  ipyparallel           Jamim       CONTRIBUTOR         1
  8   ipython  ipyparallel          ellert       CONTRIBUTOR         1
  9   ipython      ipython         Carreau            MEMBER        21
  10  ipython      ipython  LucianaMarques       CONTRIBUTOR         2
  11  ipython      ipython      OriolAbril       CONTRIBUTOR         2
  12  ipython      ipython       Pragnya02       CONTRIBUTOR         1
  13  ipython      ipython            alok       CONTRIBUTOR         2
  14  ipython      ipython         anntzer       CONTRIBUTOR         1
  15  ipython      ipython        bashtage       CONTRIBUTOR         1
  16  ipython      ipython         blueyed       CONTRIBUTOR         1
  17  ipython      ipython  brandonwillard       CONTRIBUTOR         1
  18  ipython      ipython          daharn       CONTRIBUTOR         1
  19  ipython      ipython        eobrie16       CONTRIBUTOR         1
  20  ipython      ipython      gooverdian       CONTRIBUTOR         1
  21  ipython      ipython         hroncok       CONTRIBUTOR         1
  22  ipython      ipython           k-zen       CONTRIBUTOR         1
  23  ipython      ipython          kmaork       CONTRIBUTOR         1
  24  ipython      ipython      pierstitus       CONTRIBUTOR         1
  25  ipython      ipython           rgson       CONTRIBUTOR         1
  26  ipython      ipython         sunpoet       CONTRIBUTOR         1
  27  ipython      ipython      sysadmin75              NONE         1
  28  ipython      ipython       tacaswell       CONTRIBUTOR         1
  29  ipython    traitlets           Jamim       CONTRIBUTOR         2
  30  ipython    traitlets    danielballan       CONTRIBUTOR         1
  31  ipython    traitlets   frenzymadness       CONTRIBUTOR         1
  32  ipython    traitlets     kevin-bates       CONTRIBUTOR         1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'authored',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'PR authors for ipython in the last 60 days',
  width: 1000
})

Chart({
  data:          org                repo        username authorAssociation  authored
  33   jupyter       docker-stacks    danielballan       CONTRIBUTOR         1
  34   jupyter       docker-stacks          eromoe       CONTRIBUTOR         1
  35   jupyter       docker-stacks     jamesdbrock       CONTRIBUTOR         3
  36   jupyter       docker-stacks           minrk            MEMBER         1
  37   jupyter       docker-stacks        ttimbers       CONTRIBUTOR         1
  38   jupyter  enterprise_gateway        IMAM9AIS       CONTRIBUTOR         1
  39   jupyter  enterprise_gateway       akchinSTC      COLLABORATOR         1
  40   jupyter  enterprise_gateway           clkao       CONTRIBUTOR         1
  41   jupyter  enterprise_gateway        echarles       CONTRIBUTOR         2
  42   jupyter  enterprise_gateway     kevin-bates            MEMBER        14
  43   jupyter  enterprise_gateway        lresende            MEMBER         3
  44   jupyter  enterprise_gateway  philip-p-jones       CONTRIBUTOR         1
  45   jupyter  enterprise_gateway   vikasgarg1996       CONTRIBUTOR         2
  46   jupyter          governance       ellisonbg            MEMBER         1
  47   jupyter             jupyter         Zsailer            MEMBER         1
  48   jupyter             jupyter       choldgraf            MEMBER         2
  49   jupyter        jupyter-book           cczhu       CONTRIBUTOR         1
  50   jupyter        jupyter-book       choldgraf            MEMBER         5
  51   jupyter        jupyter-book         emdupre      COLLABORATOR         1
  52   jupyter        jupyter-book            habi       CONTRIBUTOR         1
  53   jupyter        jupyter-book     thammegowda       CONTRIBUTOR         2
  54   jupyter      jupyter-sphinx        akhmerov            MEMBER         4
  55   jupyter      jupyter-sphinx      basnijholt       CONTRIBUTOR         1
  56   jupyter      jupyter-sphinx    hugokerstens       CONTRIBUTOR         1
  57   jupyter      jupyter-sphinx        jbweston            MEMBER         5
  58   jupyter   jupyter.github.io          afshin            MEMBER         2
  59   jupyter   jupyter.github.io  artur-trzesiok       CONTRIBUTOR         1
  60   jupyter   jupyter.github.io    davidanthoff       CONTRIBUTOR         1
  61   jupyter   jupyter.github.io         jMuzsik       CONTRIBUTOR         1
  62   jupyter   jupyter.github.io      jasongrout            MEMBER         1
  ..       ...                 ...             ...               ...       ...
  121  jupyter            notebook           minrk            MEMBER         1
  122  jupyter            notebook        mtd91429       CONTRIBUTOR         1
  123  jupyter            notebook        saraedum       CONTRIBUTOR         1
  124  jupyter            notebook      ssanderson       CONTRIBUTOR         1
  125  jupyter            notebook       takluyver            MEMBER         1
  126  jupyter            notebook        taohan16       CONTRIBUTOR         1
  127  jupyter            notebook          tholoz       CONTRIBUTOR         1
  128  jupyter            notebook          tmetzl       CONTRIBUTOR         1
  129  jupyter            notebook      vibvibgyor       CONTRIBUTOR         1
  130  jupyter           qtconsole           ajtam       CONTRIBUTOR         1
  131  jupyter           qtconsole      ccordoba12      COLLABORATOR         5
  132  jupyter           qtconsole        dalthviz       CONTRIBUTOR         4
  133  jupyter           qtconsole        impact27       CONTRIBUTOR         1
  134  jupyter         repo2docker      Xarthisius       CONTRIBUTOR         1
  135  jupyter         repo2docker         betatim            MEMBER        11
  136  jupyter         repo2docker       choldgraf            MEMBER         1
  137  jupyter         repo2docker    consideRatio      COLLABORATOR         1
  138  jupyter         repo2docker    davidanthoff       CONTRIBUTOR         1
  139  jupyter         repo2docker  

Chart({
  data:                  org                 repo          username authorAssociation  \
  151  jupyter-widgets           ipyleaflet         Juanlu001       CONTRIBUTOR   
  152  jupyter-widgets           ipyleaflet     SylvainCorlay            MEMBER   
  153  jupyter-widgets           ipyleaflet      VasavanThiru       CONTRIBUTOR   
  154  jupyter-widgets           ipyleaflet          deeplook       CONTRIBUTOR   
  155  jupyter-widgets           ipyleaflet             jmp75       CONTRIBUTOR   
  156  jupyter-widgets           ipyleaflet             jtpio       CONTRIBUTOR   
  157  jupyter-widgets           ipyleaflet         kgiacobbi       CONTRIBUTOR   
  158  jupyter-widgets           ipyleaflet       martinRenou            MEMBER   
  159  jupyter-widgets           ipyleaflet          mpdwulit       CONTRIBUTOR   
  160  jupyter-widgets           ipyleaflet            tgrout       CONTRIBUTOR   
  161  jupyter-widgets           ipywidgets             Jamim       CONTRIBUTOR   
  162  jupyter-widgets           ipywidgets         Juanlu001       CONTRIBUTOR   
  163  jupyter-widgets           ipywidgets     SylvainCorlay            MEMBER   
  164  jupyter-widgets           ipywidgets          akhmerov       CONTRIBUTOR   
  165  jupyter-widgets           ipywidgets             blois       CONTRIBUTOR   
  166  jupyter-widgets           ipywidgets           edolley       CONTRIBUTOR   
  167  jupyter-widgets           ipywidgets            ivanov       CONTRIBUTOR   
  168  jupyter-widgets           ipywidgets        jasongrout            MEMBER   
  169  jupyter-widgets           ipywidgets             jtpio       CONTRIBUTOR   
  170  jupyter-widgets           ipywidgets            lheagy       CONTRIBUTOR   
  171  jupyter-widgets           ipywidgets       martinRenou            MEMBER   
  172  jupyter-widgets           ipywidgets        mbektasbbg       CONTRIBUTOR   
  173  jupyter-widgets           ipywidgets           mwcraig       CONTRIBUTOR   
  174  jupyter-widgets           ipywidgets        oscar6echo       CONTRIBUTOR   
  175  jupyter-widgets           ipywidgets     throwable-one              NONE   
  176  jupyter-widgets           ipywidgets           vidartf            MEMBER   
  177  jupyter-widgets   jupyterlab-sidecar       martinRenou            MEMBER   
  178  jupyter-widgets         midicontrols             wolfv       CONTRIBUTOR   
  179  jupyter-widgets            pythreejs  GenevieveBuckley       CONTRIBUTOR   
  180  jupyter-widgets            pythreejs            ataiya       CONTRIBUTOR   
  181  jupyter-widgets            pythreejs           vidartf            MEMBER   
  182  jupyter-widgets           traittypes             Jamim       CONTRIBUTOR   
  183  jupyter-widgets             tutorial            kohr-h       CONTRIBUTOR   
  184  jupyter-widgets             tutorial   maartenbreddels            MEMBER   
  185  jupyter-widgets             tutorial       martinRenou            MEMBER   
  186  jupyter-widgets             tutorial           mwcraig       CONTRIBUTOR   
  187  jupyter-widgets  widget-cookiecutter             wolfv       CONTRIBUTOR   
  
       authored  
  151         1  
  152         4  
  153         3  
  154         1  
  155         1  
  156         1  
  157         5  
  158         6  
  159         1  
  160         1  
  161         1  
  162         1  
  163        12  
  164         1  
  165         1  
  166         1  
  167         1  
  168         3  
  169         2  
  170         1  
  171         1  
  172         1  
  173         3  
  174         1  
  175         1  
  176         1  
  177         1  
  178         1  
  179         2  
  180         1  
  181         2  
  182         1  
  183         1  
  184         2  
  185        17  
  186        10  
  187         1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR'

Chart({
  data:             org                     repo        username authorAssociation  \
  188  jupyterhub     alabaster-jupyterhub       choldgraf            MEMBER   
  189  jupyterhub             batchspawner        cmd-ntrf       CONTRIBUTOR   
  190  jupyterhub             batchspawner           kinow       CONTRIBUTOR   
  191  jupyterhub             batchspawner         rkdarst       CONTRIBUTOR   
  192  jupyterhub                   binder     jamesdbrock       CONTRIBUTOR   
  193  jupyterhub                binderhub         betatim            MEMBER   
  194  jupyterhub                binderhub          bitnik       CONTRIBUTOR   
  195  jupyterhub                binderhub       choldgraf            MEMBER   
  196  jupyterhub                binderhub         jhamman       CONTRIBUTOR   
  197  jupyterhub                binderhub           minrk            MEMBER   
  198  jupyterhub                binderhub           nuest       CONTRIBUTOR   
  199  jupyterhub                binderhub       sgibson91            MEMBER   
  200  jupyterhub                binderhub       yuvipanda            MEMBER   
  201  jupyterhub               chartpress          manics       CONTRIBUTOR   
  202  jupyterhub               chartpress       rokroskar       CONTRIBUTOR   
  203  jupyterhub            dockerspawner    danielballan       CONTRIBUTOR   
  204  jupyterhub     jupyter-server-proxy       athornton       CONTRIBUTOR   
  205  jupyterhub     jupyter-server-proxy        quasiben       CONTRIBUTOR   
  206  jupyterhub     jupyter-server-proxy          rcolle       CONTRIBUTOR   
  207  jupyterhub               jupyterhub         Carreau            MEMBER   
  208  jupyterhub               jupyterhub  GeorgianaElena       CONTRIBUTOR   
  209  jupyterhub               jupyterhub         aar0nTw       CONTRIBUTOR   
  210  jupyterhub               jupyterhub        adelcast       CONTRIBUTOR   
  211  jupyterhub               jupyterhub       bartolone              NONE   
  212  jupyterhub               jupyterhub          bitnik       CONTRIBUTOR   
  213  jupyterhub               jupyterhub        cmd-ntrf       CONTRIBUTOR   
  214  jupyterhub               jupyterhub    consideRatio            MEMBER   
  215  jupyterhub               jupyterhub       danlester       CONTRIBUTOR   
  216  jupyterhub               jupyterhub         iblis17       CONTRIBUTOR   
  217  jupyterhub               jupyterhub          ilee38       CONTRIBUTOR   
  ..          ...                      ...             ...               ...   
  245  jupyterhub             team-compass         Zsailer            MEMBER   
  246  jupyterhub             team-compass       choldgraf            MEMBER   
  247  jupyterhub             team-compass           minrk            MEMBER   
  248  jupyterhub             team-compass      ryanlovett            MEMBER   
  249  jupyterhub  the-littlest-jupyterhub  GeorgianaElena      COLLABORATOR   
  250  jupyterhub  the-littlest-jupyterhub         JuanCab       CONTRIBUTOR   
  251  jupyterhub  the-littlest-jupyterhub     cornhundred       CONTRIBUTOR   
  252  jupyterhub  the-littlest-jupyterhub           jtpio       CONTRIBUTOR   
  253  jupyterhub  the-littlest-jupyterhub         lumbric       CONTRIBUTOR   
  254  jupyterhub  the-littlest-jupyterhub           minrk            MEMBER   
  255  jupyterhub  the-littlest-jupyterhub            owah       CONTRIBUTOR   
  256  jupyterhub  the-littlest-jupyterhub        pbugnion       CONTRIBUTOR   
  257  jupyterhub  the-littlest-jupyterhub         staeiou       CONTRIBUTOR   
  258  jupyterhub  the-littlest-jupyterhub        trallard       CONTRIBUTOR   
  259  jupyterhub  the-littlest-jupyterhub       yuvipanda            MEMBER   
  260  jupyterhub            traefik-proxy  GeorgianaElena      COLLABORATOR   
  261  jupyterhub              wrapspawner        cmd-ntrf       CONTRIBUTOR   
  262  jupyterhub              yarnspawner          jcrist        

Chart({
  data:             org                           repo         username  \
  275  jupyterlab                       debugger           afshin   
  276  jupyterlab      extension-cookiecutter-js        nscozzaro   
  277  jupyterlab      extension-cookiecutter-ts          gnestor   
  278  jupyterlab      extension-cookiecutter-ts       jasongrout   
  279  jupyterlab              jupyter-renderers       dependabot   
  280  jupyterlab              jupyter-renderers         domoritz   
  281  jupyterlab              jupyter-renderers          gnestor   
  282  jupyterlab                     jupyterlab       AlbertHilb   
  283  jupyterlab                     jupyterlab           BoPeng   
  284  jupyterlab                     jupyterlab     JohanMabille   
  285  jupyterlab                     jupyterlab          Madhu94   
  286  jupyterlab                     jupyterlab          Zsailer   
  287  jupyterlab                     jupyterlab           afshin   
  288  jupyterlab                     jupyterlab       alexdetsch   
  289  jupyterlab                     jupyterlab         aschlaep   
  290  jupyterlab                     jupyterlab    balamuruganvg   
  291  jupyterlab                     jupyterlab        blink1073   
  292  jupyterlab                     jupyterlab        bobmayuze   
  293  jupyterlab                     jupyterlab   capslockwizard   
  294  jupyterlab                     jupyterlab            cnydw   
  295  jupyterlab                     jupyterlab            cquah   
  296  jupyterlab                     jupyterlab     danielballan   
  297  jupyterlab                     jupyterlab         domoritz   
  298  jupyterlab                     jupyterlab     duarteocarmo   
  299  jupyterlab                     jupyterlab        ellisonbg   
  300  jupyterlab                     jupyterlab       fcollonval   
  301  jupyterlab                     jupyterlab          gnestor   
  302  jupyterlab                     jupyterlab       ian-r-rose   
  303  jupyterlab                     jupyterlab          ibdafna   
  304  jupyterlab                     jupyterlab           ivanov   
  ..          ...                            ...              ...   
  332  jupyterlab                jupyterlab-demo    SylvainCorlay   
  333  jupyterlab                jupyterlab-demo        TheBear44   
  334  jupyterlab                jupyterlab-demo      dharmaquark   
  335  jupyterlab                jupyterlab-demo       ian-r-rose   
  336  jupyterlab                 jupyterlab-git    danielgruesso   
  337  jupyterlab                 jupyterlab-git       dependabot   
  338  jupyterlab                 jupyterlab-git       fcollonval   
  339  jupyterlab                 jupyterlab-git       jaipreet-s   
  340  jupyterlab                 jupyterlab-git     neelamgehlot   
  341  jupyterlab                 jupyterlab-git        urihoenig   
  342  jupyterlab              jupyterlab-github       dependabot   
  343  jupyterlab              jupyterlab-github       ian-r-rose   
  344  jupyterlab               jupyterlab-latex       dependabot   
  345  jupyterlab    jupyterlab-metadata-service           acu192   
  346  jupyterlab           jupyterlab-telemetry        yuvipanda   
  347  jupyterlab                 jupyterlab-toc       dependabot   
  348  jupyterlab                 jupyterlab-toc        marius311   
  349  jupyterlab            jupyterlab_pygments  maartenbreddels   
  350  jupyterlab              jupyterlab_server           afshin   
  351  jupyterlab              jupyterlab_server        blink1073   
  352  jupyterlab              jupyterlab_server       ian-r-rose   
  353  jupyterlab              jupyterlab_server       telamonian   
  354  jupyterlab              jupyterlab_server        thomafred   
  355  jupyterlab              jupyterlab_server          zerline   
  356  jupyterlab     mimerender-cookiecutter-ts       jasongrout   
  357  jupyterlab  scipy2019-jupyterlab-tutorial     

Chart({
  data:          org            repo        username authorAssociation  authored
  362  nteract       bookstore          mpacer            MEMBER        19
  363  nteract       bookstore        willingc            MEMBER        17
  364  nteract        commuter      dependabot       CONTRIBUTOR         2
  365  nteract        commuter          taniki            MEMBER         1
  366  nteract        hydrogen      BenRussert            MEMBER         4
  367  nteract        hydrogen        aviatesk            MEMBER         9
  368  nteract        hydrogen        hoishing            MEMBER         1
  369  nteract        hydrogen       jwrussert       CONTRIBUTOR         1
  370  nteract        hydrogen        renovate       CONTRIBUTOR         6
  371  nteract        hydrogen  wadethestealth            MEMBER        18
  372  nteract    kernel-relay      BenRussert            MEMBER         1
  373  nteract  minimal-plotly        renovate              NONE         2
  374  nteract         nteract      BenRussert            MEMBER         7
  375  nteract         nteract  CrystallineCat            MEMBER         2
  376  nteract         nteract         and0111       CONTRIBUTOR         1
  377  nteract         nteract    captainsafia            MEMBER         7
  378  nteract         nteract      dylanross3            MEMBER         1
  379  nteract         nteract        renovate       CONTRIBUTOR        19
  380  nteract         nteract        rganardi            MEMBER         2
  381  nteract         nteract  wadethestealth            MEMBER         1
  382  nteract      nteract.io        renovate              NONE         9
  383  nteract       papermill           MSeal            MEMBER         2
  384  nteract       papermill     RoelBrouwer       CONTRIBUTOR         1
  385  nteract       papermill  VincentBLortie            MEMBER         1
  386  nteract       papermill   aabdullah-bos            MEMBER         1
  387  nteract       papermill       casperdcl            MEMBER         3
  388  nteract       papermill    duarteocarmo            MEMBER         3
  389  nteract       papermill        khourhin            MEMBER         1
  390  nteract       papermill          mwouts            MEMBER         1
  391  nteract       papermill          rgbkrk            MEMBER         1
  392  nteract       papermill        trallard            MEMBER         1
  393  nteract       papermill          vizhur       CONTRIBUTOR         1
  394  nteract       scrapbook           MSeal            MEMBER         1
  395  nteract       scrapbook      matthiasdv            MEMBER         1
  396  nteract       scrapbook        trallard            MEMBER         1
  397  nteract            vdom         gnestor            MEMBER         1
  398  nteract   vega-embed-v3  CrystallineCat            MEMBER         2,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'authored',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'PR authors for nteract in the last 60 days',
  width: 1000
})

In [143]:
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    display(ch)

Chart({
  data:                org                repo   username  closed
  0  binder-examples          data-quilt    betatim       1
  1  binder-examples  getting-data-xroot  choldgraf       1
  2  binder-examples               latex  choldgraf       1,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:        org         repo  username  closed
  3  ipython    ipykernel     minrk       4
  4  ipython  ipyparallel     minrk       2
  5  ipython      ipython   Carreau      43
  6  ipython    traitlets  rmorshea       5,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for ipython in the last 60 days',
  width: 1000
})

Chart({
  data:         org                repo       username  closed
  7   jupyter       docker-stacks          minrk       6
  8   jupyter       docker-stacks        parente       1
  9   jupyter  enterprise_gateway      akchinSTC       2
  10  jupyter  enterprise_gateway    kevin-bates      17
  11  jupyter  enterprise_gateway       lresende       6
  12  jupyter          governance      choldgraf       1
  13  jupyter             jupyter      choldgraf       2
  14  jupyter             jupyter        parente       1
  15  jupyter        jupyter-book      choldgraf      10
  16  jupyter      jupyter-sphinx  SylvainCorlay       1
  17  jupyter      jupyter-sphinx       akhmerov       9
  18  jupyter      jupyter-sphinx       jbweston       1
  19  jupyter   jupyter.github.io        Carreau       1
  20  jupyter   jupyter.github.io           Ruv7       1
  21  jupyter   jupyter.github.io         afshin       1
  22  jupyter   jupyter.github.io      choldgraf       2
  23  jupyter   jupyter.github.io      ellisonbg       1
  24  jupyter   jupyter.github.io     jasongrout       2
  25  jupyter      jupyter_client          MSeal       1
  26  jupyter      jupyter_client  SylvainCorlay       4
  27  jupyter      jupyter_client      blink1073       1
  28  jupyter      jupyter_client          minrk       1
  29  jupyter      jupyter_client         mpacer       1
  30  jupyter      jupyter_client         rgbkrk       4
  31  jupyter        jupyter_core          minrk       1
  32  jupyter        jupyter_core         rgbkrk       1
  33  jupyter        jupyter_core      takluyver       5
  34  jupyter      jupyter_server  SylvainCorlay       3
  35  jupyter      jupyter_server      blink1073       1
  36  jupyter      jupyter_server         ivanov       1
  37  jupyter      jupyter_server       rolweber       3
  38  jupyter               nb2kg    kevin-bates       2
  39  jupyter           nbconvert          MSeal      11
  40  jupyter           nbconvert       t-makaro       4
  41  jupyter           nbconvert       willingc       1
  42  jupyter              nbdime        vidartf       1
  43  jupyter            nbgrader       jhamrick      43
  44  jupyter            nbviewer      choldgraf       1
  45  jupyter            nbviewer        parente       3
  46  jupyter            notebook  SylvainCorlay       2
  47  jupyter            notebook         ivanov       1
  48  jupyter            notebook       lresende       9
  49  jupyter            notebook          minrk       4
  50  jupyter            notebook      takluyver      32
  51  jupyter           qtconsole     ccordoba12      11
  52  jupyter         repo2docker        betatim      19
  53  jupyter         repo2docker      choldgraf       1
  54  jupyter         repo2docker          minrk       5
  55  jupyter         repo2docker      yuvipanda       5
  56  jupyter             roadmap      choldgraf       1
  57  jupyter           telemetry        Zsailer       6
  58  jupyter           telemetry     jaipreet-s       2
  59  jupyter           telemetry      yuvipanda       7,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for jupyter in the last 60 days',
  width: 1000
})

Chart({
  data:                 org                 repo         username  closed
  60  jupyter-widgets           ipyleaflet    SylvainCorlay       3
  61  jupyter-widgets           ipyleaflet      martinRenou      21
  62  jupyter-widgets           ipywidgets    SylvainCorlay      22
  63  jupyter-widgets           ipywidgets       jasongrout       7
  64  jupyter-widgets           ipywidgets          vidartf       3
  65  jupyter-widgets   jupyterlab-sidecar    SylvainCorlay       1
  66  jupyter-widgets         midicontrols       jasongrout       1
  67  jupyter-widgets            pythreejs          vidartf       5
  68  jupyter-widgets           traittypes    SylvainCorlay       1
  69  jupyter-widgets             tutorial    SylvainCorlay       1
  70  jupyter-widgets             tutorial  maartenbreddels       1
  71  jupyter-widgets             tutorial      martinRenou       4
  72  jupyter-widgets             tutorial          mwcraig      24
  73  jupyter-widgets  widget-cookiecutter          vidartf       1,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for jupyter-widgets in the last 60 days',
  width: 1000
})

Chart({
  data:             org                     repo        username  closed
  74   jupyterhub     alabaster-jupyterhub         betatim       1
  75   jupyterhub     alabaster-jupyterhub       choldgraf       8
  76   jupyterhub             batchspawner      mbmilligan       7
  77   jupyterhub                   binder         betatim       1
  78   jupyterhub                binderhub         betatim      12
  79   jupyterhub                binderhub       choldgraf       2
  80   jupyterhub                binderhub    consideRatio       3
  81   jupyterhub                binderhub           minrk       1
  82   jupyterhub                binderhub       sgibson91       1
  83   jupyterhub                binderhub       yuvipanda       1
  84   jupyterhub               chartpress           minrk       1
  85   jupyterhub               chartpress       yuvipanda       1
  86   jupyterhub            dockerspawner           minrk       1
  87   jupyterhub     jupyter-server-proxy         betatim       1
  88   jupyterhub     jupyter-server-proxy       yuvipanda       2
  89   jupyterhub               jupyterhub         betatim       4
  90   jupyterhub               jupyterhub           minrk      10
  91   jupyterhub               jupyterhub        willingc       4
  92   jupyterhub               jupyterhub       yuvipanda       3
  93   jupyterhub    kerberosauthenticator          jcrist       4
  94   jupyterhub         ltiauthenticator       yuvipanda       1
  95   jupyterhub      mybinder.org-deploy         betatim      57
  96   jupyterhub      mybinder.org-deploy       choldgraf       3
  97   jupyterhub      mybinder.org-deploy          henchc       1
  98   jupyterhub      mybinder.org-deploy           minrk       1
  99   jupyterhub      mybinder.org-deploy       sgibson91       1
  100  jupyterhub              nbgitpuller       choldgraf       2
  101  jupyterhub              nbgitpuller       yuvipanda       3
  102  jupyterhub           oauthenticator         betatim       1
  103  jupyterhub           oauthenticator           minrk       3
  104  jupyterhub              sudospawner    consideRatio       1
  105  jupyterhub           systemdspawner       yuvipanda       1
  106  jupyterhub             team-compass         betatim       4
  107  jupyterhub             team-compass       choldgraf       4
  108  jupyterhub             team-compass       yuvipanda       1
  109  jupyterhub  the-littlest-jupyterhub  GeorgianaElena       5
  110  jupyterhub  the-littlest-jupyterhub         betatim       7
  111  jupyterhub  the-littlest-jupyterhub       choldgraf       4
  112  jupyterhub  the-littlest-jupyterhub           minrk       1
  113  jupyterhub  the-littlest-jupyterhub       yuvipanda      23
  114  jupyterhub            traefik-proxy           minrk       5
  115  jupyterhub              wrapspawner      mbmilligan       2
  116  jupyterhub              yarnspawner          jcrist       3
  117  jupyterhub   zero-to-jupyterhub-k8s         betatim       1
  118  jupyterhub   zero-to-jupyterhub-k8s    consideRatio       9
  119  jupyterhub   zero-to-jupyterhub-k8s           minrk       2,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for jupyterhub in the last 60 days',
  width: 1000
})

Chart({
  data:             org                           repo        username  closed
  120  jupyterlab                       debugger          afshin       1
  121  jupyterlab      extension-cookiecutter-js           jtpio       1
  122  jupyterlab      extension-cookiecutter-ts       blink1073       2
  123  jupyterlab              jupyter-renderers         gnestor       5
  124  jupyterlab              jupyter-renderers      ian-r-rose       2
  125  jupyterlab                     jupyterlab          afshin       6
  126  jupyterlab                     jupyterlab        aschlaep       2
  127  jupyterlab                     jupyterlab       blink1073      61
  128  jupyterlab                     jupyterlab       ellisonbg       5
  129  jupyterlab                     jupyterlab         gnestor       2
  130  jupyterlab                     jupyterlab      ian-r-rose      20
  131  jupyterlab                     jupyterlab          ivanov       2
  132  jupyterlab                     jupyterlab      jasongrout      56
  133  jupyterlab                     jupyterlab           jtpio       1
  134  jupyterlab                     jupyterlab  saulshanabrook       5
  135  jupyterlab                     jupyterlab      telamonian       3
  136  jupyterlab                     jupyterlab         vidartf       2
  137  jupyterlab          jupyterlab-commenting          acu192       1
  138  jupyterlab          jupyterlab-commenting          hoo761       4
  139  jupyterlab       jupyterlab-data-explorer  saulshanabrook      10
  140  jupyterlab                jupyterlab-demo     dharmaquark       4
  141  jupyterlab                jupyterlab-demo      ian-r-rose       3
  142  jupyterlab                jupyterlab-demo      jasongrout       1
  143  jupyterlab                 jupyterlab-git           dleen       3
  144  jupyterlab                 jupyterlab-git      jaipreet-s       9
  145  jupyterlab                 jupyterlab-git  saulshanabrook       1
  146  jupyterlab              jupyterlab-github      ian-r-rose       2
  147  jupyterlab               jupyterlab-latex      ian-r-rose       1
  148  jupyterlab    jupyterlab-metadata-service          acu192       1
  149  jupyterlab           jupyterlab-telemetry         Zsailer       1
  150  jupyterlab                 jupyterlab-toc      ian-r-rose       1
  151  jupyterlab                 jupyterlab-toc      jasongrout       1
  152  jupyterlab            jupyterlab_pygments   SylvainCorlay       1
  153  jupyterlab              jupyterlab_server          afshin       4
  154  jupyterlab              jupyterlab_server       blink1073       5
  155  jupyterlab     mimerender-cookiecutter-ts      jasongrout       1
  156  jupyterlab  scipy2019-jupyterlab-tutorial         Carreau       2
  157  jupyterlab             theme-cookiecutter       blink1073       3,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for jupyterlab in the last 60 days',
  width: 1000
})

Chart({
  data:          org            repo        username  closed
  158  nteract       bookstore           MSeal       3
  159  nteract       bookstore          mpacer       8
  160  nteract       bookstore          rgbkrk       6
  161  nteract       bookstore        willingc      19
  162  nteract        commuter    captainsafia       3
  163  nteract        hydrogen      BenRussert      21
  164  nteract        hydrogen        aviatesk      10
  165  nteract        hydrogen        renovate       4
  166  nteract        hydrogen  wadethestealth       4
  167  nteract    kernel-relay    captainsafia       1
  168  nteract  minimal-plotly          rgbkrk       2
  169  nteract         nteract      BenRussert       7
  170  nteract         nteract  CrystallineCat       2
  171  nteract         nteract    captainsafia      21
  172  nteract         nteract          rgbkrk      10
  173  nteract      nteract.io        renovate       8
  174  nteract      nteract.io          rgbkrk       1
  175  nteract       papermill           MSeal      11
  176  nteract       papermill          rgbkrk       4
  177  nteract       papermill        willingc       1
  178  nteract       scrapbook           MSeal       2
  179  nteract       scrapbook          rgbkrk       1
  180  nteract            vdom         gnestor       1
  181  nteract   vega-embed-v3  CrystallineCat       2,
  encoding: FacetedEncoding({
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'closed',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for nteract in the last 60 days',
  width: 1000
})

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [144]:
responses = []
for org in github_orgs:
    query_prs = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
created = pd.concat([ii.data for ii in responses])
created['n_comments'] = created['comments'].map(lambda a: len(a['edges']))
created = created.query('username not in @bot_names')

Found 195 items, which will take 4 pages
Found 191 items, which will take 4 pages
Found 210 items, which will take 5 pages
Found 55 items, which will take 2 pages
Found 37 items, which will take 1 pages
Found 1 items, which will take 1 pages
Found 83 items, which will take 2 pages


In [145]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0   binder-examples                 jupyter-rise       1   
  1           ipython                    ipykernel       6   
  2           ipython                  ipyparallel       3   
  3           ipython                      ipython      27   
  4           ipython             ipython-in-depth       1   
  5           jupyter                accessibility       1   
  6           jupyter                       design       1   
  7           jupyter                docker-stacks      15   
  8           jupyter           enterprise_gateway       6   
  9           jupyter                      jupyter       4   
  10          jupyter                 jupyter-book      17   
  11          jupyter                jupyter-drive       1   
  12          jupyter               jupyter-sphinx       7   
  13          jupyter               jupyter_client       5   
  14          jupyter                 jupyter_core       1   
  15          jupyter          jupyter_kernel_test       1   
  16          jupyter               jupyter_server       8   
  17          jupyter               kernel_gateway       1   
  18          jupyter                        nb2kg       2   
  19          jupyter                    nbconvert      11   
  20          jupyter                       nbdime       6   
  21          jupyter                     nbgrader      24   
  22          jupyter                     nbviewer       3   
  23          jupyter                     notebook      51   
  24          jupyter                    qtconsole       3   
  25          jupyter                  repo2docker      22   
  26          jupyter                     testpath       1   
  27  jupyter-widgets                   ipyleaflet       7   
  28  jupyter-widgets                   ipywidgets      38   
  29  jupyter-widgets           jupyterlab-sidecar       3   
  ..              ...                          ...     ...   
  57       jupyterhub                  wrapspawner       3   
  58       jupyterhub       zero-to-jupyterhub-k8s      23   
  59       jupyterlab    extension-cookiecutter-ts       1   
  60       jupyterlab            jupyter-renderers       2   
  61       jupyterlab                   jupyterlab     156   
  62       jupyterlab          jupyterlab-celltags       3   
  63       jupyterlab        jupyterlab-commenting      12   
  64       jupyterlab     jupyterlab-data-explorer      19   
  65       jupyterlab               jupyterlab-git       7   
  66       jupyterlab      jupyterlab-google-drive       1   
  67       jupyterlab  jupyterlab-metadata-service       1   
  68       jupyterlab         jupyterlab-telemetry       2   
  69       jupyterlab               jupyterlab-toc       1   
  70       jupyterlab              jupyterlab_apod       1   
  71       jupyterlab               jupyterlab_app       1   
  72       jupyterlab                 team-compass       3   
  73          nteract                    bookstore      13   
  74          nteract                  coffee_boat       1   
  75          nteract                     commuter       1   
  76          nteract           commuter-on-glitch       1   
  77          nteract                           dx       3   
  78          nteract                     hydrogen      13   
  79          nteract              meeting-minutes       1   
  80          nteract               minimal-plotly       1   
  81          nteract                      nteract      24   
  82          nteract                   nteract.io       1   
  83          nteract                    papermill      13   
  84          nteract                    scrapbook       3   
  85          nteract                     semiotic       7   
  86          nteract                         vdom       1   
  
                                    org/repo  
  0             binder-examples/jupyter-rise  
  1                        ipython/ipykernel 

In [146]:
responses = []
for org in github_orgs:
    query_prs = f"is:issue user:{org} closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
closed = pd.concat([ii.data for ii in responses])
closed['n_comments'] = closed['comments'].map(lambda a: len(a['edges']))
closed = closed.query('username not in @bot_names')

Found 141 items, which will take 3 pages
Found 219 items, which will take 5 pages
Found 221 items, which will take 5 pages
Found 53 items, which will take 2 pages
Found 36 items, which will take 1 pages
Found 2 items, which will take 1 pages
Found 174 items, which will take 4 pages


In [147]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                      bokeh       1   
  1   binder-examples                 demo-julia       1   
  2           ipython                  ipykernel       2   
  3           ipython                    ipython      32   
  4           ipython                  traitlets       2   
  5           jupyter              atom-notebook       1   
  6           jupyter              docker-stacks       7   
  7           jupyter         enterprise_gateway      21   
  8           jupyter                    jupyter       3   
  9           jupyter               jupyter-book      11   
  10          jupyter             jupyter-sphinx      13   
  11          jupyter          jupyter.github.io       1   
  12          jupyter             jupyter_client       4   
  13          jupyter               jupyter_core       9   
  14          jupyter             jupyter_server       3   
  15          jupyter                      nb2kg       2   
  16          jupyter                  nbconvert      18   
  17          jupyter                     nbdime       2   
  18          jupyter                   nbgrader      39   
  19          jupyter                   nbviewer       4   
  20          jupyter                   notebook      54   
  21          jupyter                  qtconsole       6   
  22          jupyter                repo2docker      17   
  23          jupyter                  telemetry       3   
  24          jupyter                  terminado       1   
  25  jupyter-widgets                 ipyleaflet      14   
  26  jupyter-widgets                 ipywidgets      31   
  27  jupyter-widgets         jupyterlab-sidecar       1   
  28  jupyter-widgets               midicontrols       1   
  29  jupyter-widgets                  pythreejs       3   
  ..              ...                        ...     ...   
  35       jupyterhub       jupyter-server-proxy       1   
  36       jupyterhub                 jupyterhub      22   
  37       jupyterhub   jupyterhub-deploy-docker       1   
  38       jupyterhub      kerberosauthenticator       1   
  39       jupyterhub                kubespawner       2   
  40       jupyterhub        mybinder.org-deploy      11   
  41       jupyterhub                nbgitpuller      10   
  42       jupyterhub             oauthenticator       1   
  43       jupyterhub               team-compass      17   
  44       jupyterhub    the-littlest-jupyterhub      38   
  45       jupyterhub              traefik-proxy       5   
  46       jupyterhub                yarnspawner       1   
  47       jupyterhub     zero-to-jupyterhub-k8s      15   
  48       jupyterlab  extension-cookiecutter-js       1   
  49       jupyterlab          jupyter-renderers       7   
  50       jupyterlab                 jupyterlab     186   
  51       jupyterlab   jupyterlab-data-explorer       9   
  52       jupyterlab            jupyterlab-demo       2   
  53       jupyterlab             jupyterlab-git      12   
  54       jupyterlab          jupyterlab-github       1   
  55       jupyterlab             jupyterlab-toc       2   
  56       jupyterlab          jupyterlab_server       1   
  57          nteract                  bookstore      11   
  58          nteract                   commuter       1   
  59          nteract                   hydrogen     103   
  60          nteract                    nteract      39   
  61          nteract                  papermill      11   
  62          nteract                  scrapbook       1   
  63          nteract                   semiotic       5   
  64          nteract                       vdom       3   
  
                                  org/repo  
  0                  binder-examples/bokeh  
  1             binder-examples/demo-julia  
  2                      ipython/ipykernel  
  3                        ipython/ipython  
  4                      ipython/trait

### A list of recent issues

In [148]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, issue in issues.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['username']
            user_url = issue['username_url']
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 60 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [149]:
comments = []
for items in [created, merged, closed]:
    for ii, irow in items.iterrows():
        icomments = [ii['node'] for ii in irow['comments']['edges']]
        icomments = pd.DataFrame(icomments)
        icomments['repo'] = irow['repo']
        icomments['org'] = irow['org']
        comments.append(icomments)
comments = pd.concat(comments, sort=False)
comments = comments.dropna(subset=['author'])
comments = comments.query("createdAt > @start_date and createdAt < @end_date")
comments['username'] = comments['author'].map(lambda a: a['login'])
comments['username_url'] = comments['author'].map(lambda a: a['url'])
comments = comments.query('username not in @bot_names')

In [150]:
commentors = (
    comments
    .groupby(['org', 'repo', 'username', 'authorAssociation'])
    .count()['author']
    .reset_index()
    .sort_values(['org', 'author'], ascending=False)
)

In [151]:
n_plot = 50
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='username',
        y='author',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org                repo             username authorAssociation  \
  0  binder-examples               bokeh            choldgraf            MEMBER   
  4  binder-examples          demo-julia            choldgraf            MEMBER   
  1  binder-examples               bokeh  mathematicalmichael              NONE   
  2  binder-examples          data-quilt              betatim            MEMBER   
  3  binder-examples          data-quilt          matheusmota              NONE   
  5  binder-examples          demo-julia         davidanthoff       CONTRIBUTOR   
  6  binder-examples  getting-data-xroot            choldgraf            MEMBER   
  7  binder-examples  getting-data-xroot      matthewfeickert       CONTRIBUTOR   
  8  binder-examples        jupyter-rise            choldgraf            MEMBER   
  9  binder-examples        jupyter-rise  mathematicalmichael              NONE   
  
     author  
  0       2  
  4       2  
  1       1  
  2       1  
  3       1  
  5       1  
  6       1  
  7       1  
  8       1  
  9       1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org         repo        username authorAssociation  author
  19  ipython      ipython         Carreau            MEMBER      83
  13  ipython    ipykernel         hroncok       CONTRIBUTOR       7
  65  ipython    traitlets        rmorshea      COLLABORATOR       6
  15  ipython    ipykernel           minrk            MEMBER       5
  20  ipython      ipython  LucianaMarques       CONTRIBUTOR       5
  12  ipython    ipykernel      ccordoba12       CONTRIBUTOR       4
  36  ipython      ipython         efiring       CONTRIBUTOR       4
  54  ipython      ipython       tacaswell       CONTRIBUTOR       3
  61  ipython    traitlets     kevin-bates       CONTRIBUTOR       3
  10  ipython    ipykernel         Carreau            MEMBER       2
  17  ipython  ipyparallel           minrk            MEMBER       2
  27  ipython      ipython         blueyed       CONTRIBUTOR       2
  35  ipython      ipython          dlukes              NONE       2
  38  ipython      ipython        fehiepsi              NONE       2
  44  ipython      ipython      jamesmyatt              NONE       2
  45  ipython      ipython        jfhbrook              NONE       2
  47  ipython      ipython        jseabold       CONTRIBUTOR       2
  49  ipython      ipython          kmaork       CONTRIBUTOR       2
  55  ipython      ipython      thrasibule              NONE       2
  56  ipython      ipython        voronind              NONE       2
  57  ipython      ipython         wenjuno              NONE       2
  59  ipython    traitlets           Jamim       CONTRIBUTOR       2
  64  ipython    traitlets          rgbkrk            MEMBER       2
  11  ipython    ipykernel         Chilipp       CONTRIBUTOR       1
  14  ipython    ipykernel      jamesmyatt              NONE       1
  16  ipython  ipyparallel           Jamim       CONTRIBUTOR       1
  18  ipython      ipython    Benjamin-Lee              NONE       1
  21  ipython      ipython      OriolAbril       CONTRIBUTOR       1
  22  ipython      ipython        aakhmetz              NONE       1
  23  ipython      ipython    alessioarena              NONE       1
  24  ipython      ipython      alexmojaki              NONE       1
  25  ipython      ipython            alok       CONTRIBUTOR       1
  26  ipython      ipython         anntzer       CONTRIBUTOR       1
  28  ipython      ipython  brandonwillard       CONTRIBUTOR       1
  29  ipython      ipython    brunobeltran              NONE       1
  30  ipython      ipython      ccordoba12       CONTRIBUTOR       1
  31  ipython      ipython         csheaff              NONE       1
  32  ipython      ipython          daharn       CONTRIBUTOR       1
  33  ipython      ipython       davidnero              NONE       1
  34  ipython      ipython         dickmao              NONE       1
  37  ipython      ipython  farisachugthai              NONE       1
  39  ipython      ipython       gdhameeja              NONE       1
  40  ipython      ipython        gpotter2       CONTRIBUTOR       1
  41  ipython      ipython        henri-kt              NONE       1
  42  ipython      ipython         hroncok       CONTRIBUTOR       1
  43  ipython      ipython      ikizhvatov              NONE       1
  46  ipython      ipython  joaomarcosgris              NONE       1
  48  ipython      ipython           kernc              NONE       1
  50  ipython      ipython           kmtac              NONE       1
  51  ipython      ipython         scopatz       CONTRIBUTOR       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      

Chart({
  data:          org                repo       username authorAssociation  author
  203  jupyter            nbgrader       jhamrick            MEMBER      98
  298  jupyter         repo2docker        betatim            MEMBER      85
  277  jupyter            notebook      takluyver            MEMBER      65
  90   jupyter  enterprise_gateway    kevin-bates            MEMBER      56
  119  jupyter      jupyter-sphinx       akhmerov            MEMBER      40
  106  jupyter        jupyter-book      choldgraf            MEMBER      39
  170  jupyter           nbconvert          MSeal            MEMBER      38
  320  jupyter         repo2docker      yuvipanda            MEMBER      30
  156  jupyter      jupyter_server        Zsailer            MEMBER      25
  301  jupyter         repo2docker   consideRatio      COLLABORATOR      23
  310  jupyter         repo2docker          minrk            MEMBER      22
  321  jupyter           telemetry        Zsailer            MEMBER      20
  137  jupyter      jupyter_client  SylvainCorlay            MEMBER      19
  285  jupyter           qtconsole     ccordoba12      COLLABORATOR      17
  324  jupyter           telemetry      yuvipanda            MEMBER      17
  155  jupyter      jupyter_server  SylvainCorlay            MEMBER      16
  207  jupyter            nbgrader        nthiery       CONTRIBUTOR      16
  211  jupyter            nbgrader        rkdarst       CONTRIBUTOR      16
  182  jupyter           nbconvert       t-makaro      COLLABORATOR      15
  219  jupyter            nbviewer       krinsman       CONTRIBUTOR      15
  322  jupyter           telemetry     jaipreet-s      COLLABORATOR      14
  86   jupyter  enterprise_gateway       echarles       CONTRIBUTOR      13
  162  jupyter      jupyter_server       rolweber      COLLABORATOR      12
  300  jupyter         repo2docker      choldgraf            MEMBER      12
  95   jupyter  enterprise_gateway       suryag10              NONE      11
  136  jupyter      jupyter_client          MSeal            MEMBER      11
  260  jupyter            notebook  joshuazeltser       CONTRIBUTOR      11
  266  jupyter            notebook       lresende            MEMBER      10
  275  jupyter            notebook     ssanderson       CONTRIBUTOR      10
  302  jupyter         repo2docker   davidanthoff       CONTRIBUTOR      10
  118  jupyter      jupyter-sphinx  SylvainCorlay            MEMBER       9
  153  jupyter        jupyter_core      takluyver            MEMBER       9
  169  jupyter               nb2kg    kevin-bates            MEMBER       9
  193  jupyter            nbgrader        Sefriol              NONE       9
  78   jupyter       docker-stacks        parente            MEMBER       8
  108  jupyter        jupyter-book        emdupre      COLLABORATOR       8
  210  jupyter            nbgrader    psychemedia              NONE       8
  124  jupyter      jupyter-sphinx       jbweston            MEMBER       7
  190  jupyter              nbdime        vidartf      COLLABORATOR       7
  198  jupyter            nbgrader        dsblank            MEMBER       7
  199  jupyter            nbgrader     gertingold       CONTRIBUTOR       7
  87   jupyter  enterprise_gateway         esevan      COLLABORATOR       6
  122  jupyter      jupyter-sphinx   hugokerstens       CONTRIBUTOR       6
  147  jupyter      jupyter_client         rgbkrk            MEMBER       6
  221  jupyter            nbviewer        parente            MEMBER       6
  235  jupyter            notebook  SylvainCorlay            MEMBER       6
  271  jupyter            notebook         mpacer            MEMBER       6
  303  jupyter         repo2docker     davidrpugh       CONTRIBUTOR       6
  82   jupyter  enterprise_gateway       IMAM9AIS       CONTRIBUTOR       5
  101  jupyter             jupyter      choldgraf            MEMBER       5,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
     

Chart({
  data:                  org                repo          username authorAssociation  \
  347  jupyter-widgets          ipywidgets     SylvainCorlay            MEMBER   
  356  jupyter-widgets          ipywidgets        jasongrout            MEMBER   
  369  jupyter-widgets          ipywidgets           vidartf            MEMBER   
  335  jupyter-widgets          ipyleaflet       martinRenou            MEMBER   
  390  jupyter-widgets            tutorial           mwcraig       CONTRIBUTOR   
  383  jupyter-widgets           pythreejs           vidartf            MEMBER   
  354  jupyter-widgets          ipywidgets             hainm              NONE   
  338  jupyter-widgets          ipyleaflet          stonebig       CONTRIBUTOR   
  389  jupyter-widgets            tutorial       martinRenou            MEMBER   
  362  jupyter-widgets          ipywidgets           mwcraig       CONTRIBUTOR   
  334  jupyter-widgets          ipyleaflet         kgiacobbi       CONTRIBUTOR   
  346  jupyter-widgets          ipywidgets         Ricyteach       CONTRIBUTOR   
  378  jupyter-widgets           pythreejs          aliddell              NONE   
  333  jupyter-widgets          ipyleaflet             jtpio       CONTRIBUTOR   
  358  jupyter-widgets          ipywidgets           kafonek              NONE   
  359  jupyter-widgets          ipywidgets   maartenbreddels            MEMBER   
  365  jupyter-widgets          ipywidgets        rskabelund              NONE   
  388  jupyter-widgets            tutorial   maartenbreddels            MEMBER   
  351  jupyter-widgets          ipywidgets         choldgraf              NONE   
  366  jupyter-widgets          ipywidgets        ssanderson       CONTRIBUTOR   
  370  jupyter-widgets          ipywidgets       xilunaJumma              NONE   
  374  jupyter-widgets  jupyterlab-sidecar        doutriaux1              NONE   
  381  jupyter-widgets           pythreejs          stonebig              NONE   
  329  jupyter-widgets          ipyleaflet     SylvainCorlay            MEMBER   
  330  jupyter-widgets          ipyleaflet          chbrandt              NONE   
  331  jupyter-widgets          ipyleaflet          deeplook       CONTRIBUTOR   
  332  jupyter-widgets          ipyleaflet       dwr-psandhu              NONE   
  340  jupyter-widgets          ipywidgets   DanielAristidou              NONE   
  367  jupyter-widgets          ipywidgets          stonebig       CONTRIBUTOR   
  372  jupyter-widgets  jupyterlab-sidecar     SylvainCorlay            MEMBER   
  376  jupyter-widgets        midicontrols        jasongrout            MEMBER   
  391  jupyter-widgets            tutorial           vidartf            MEMBER   
  328  jupyter-widgets          ipyleaflet         Juanlu001       CONTRIBUTOR   
  336  jupyter-widgets          ipyleaflet          mpdwulit       CONTRIBUTOR   
  337  jupyter-widgets          ipyleaflet       paulathomas              NONE   
  339  jupyter-widgets          ipywidgets        DYSpider13              NONE   
  341  jupyter-widgets          ipywidgets          DevFabrx              NONE   
  342  jupyter-widgets          ipywidgets    IlyaKazakevich       CONTRIBUTOR   
  343  jupyter-widgets          ipywidgets         JohnIssue              NONE   
  344  jupyter-widgets          ipywidgets         Juanlu001       CONTRIBUTOR   
  345  jupyter-widgets          ipywidgets         Liwellyen              NONE   
  348  jupyter-widgets          ipywidgets          akhmerov       CONTRIBUTOR   
  349  jupyter-widgets          ipywidgets    artur-trzesiok              NONE   
  350  jupyter-widgets          ipywidgets            bybyte              NONE   
  352  jupyter-widgets          ipywidgets       cornhundred       CONTRIBUTOR   
  353  jupyter-widgets          ipywidgets          dazzag24              NONE   
  355  jupyter-widgets          ipywidgets            ivanov       CONTRIBUTOR   
  357  jupyter-widgets          ipywidgets       j

Chart({
  data:             org                     repo         username authorAssociation  \
  581  jupyterhub  the-littlest-jupyterhub        yuvipanda            MEMBER   
  501  jupyterhub      mybinder.org-deploy          betatim            MEMBER   
  542  jupyterhub             team-compass        choldgraf            MEMBER   
  404  jupyterhub                binderhub          betatim            MEMBER   
  541  jupyterhub             team-compass          betatim            MEMBER   
  520  jupyterhub              nbgitpuller        yuvipanda            MEMBER   
  601  jupyterhub   zero-to-jupyterhub-k8s     consideRatio            MEMBER   
  509  jupyterhub      mybinder.org-deploy      mael-le-gal            MEMBER   
  477  jupyterhub               jupyterhub          ryogesh       CONTRIBUTOR   
  503  jupyterhub      mybinder.org-deploy        choldgraf            MEMBER   
  557  jupyterhub  the-littlest-jupyterhub   GeorgianaElena      COLLABORATOR   
  399  jupyterhub             batchspawner          rkdarst       CONTRIBUTOR   
  406  jupyterhub                binderhub        choldgraf            MEMBER   
  551  jupyterhub             team-compass        sgibson91            MEMBER   
  471  jupyterhub               jupyterhub            minrk            MEMBER   
  549  jupyterhub             team-compass            minrk            MEMBER   
  514  jupyterhub              nbgitpuller        choldgraf            MEMBER   
  585  jupyterhub            traefik-proxy            minrk            MEMBER   
  407  jupyterhub                binderhub     consideRatio            MEMBER   
  512  jupyterhub      mybinder.org-deploy        yuvipanda            MEMBER   
  517  jupyterhub              nbgitpuller      parmentelat       CONTRIBUTOR   
  419  jupyterhub                binderhub        sgibson91            MEMBER   
  553  jupyterhub             team-compass        yuvipanda            MEMBER   
  486  jupyterhub    kerberosauthenticator           jcrist            MEMBER   
  583  jupyterhub            traefik-proxy   GeorgianaElena      COLLABORATOR   
  609  jupyterhub   zero-to-jupyterhub-k8s           manics       CONTRIBUTOR   
  441  jupyterhub               jupyterhub   GeorgianaElena       CONTRIBUTOR   
  445  jupyterhub               jupyterhub  RandyBetancourt              NONE   
  480  jupyterhub               jupyterhub        vilhelmen       CONTRIBUTOR   
  490  jupyterhub              kubespawner     consideRatio            MEMBER   
  546  jupyterhub             team-compass           jcrist            MEMBER   
  607  jupyterhub   zero-to-jupyterhub-k8s         joshbode       CONTRIBUTOR   
  397  jupyterhub             batchspawner       mbmilligan            MEMBER   
  482  jupyterhub               jupyterhub        yuvipanda            MEMBER   
  510  jupyterhub      mybinder.org-deploy            minrk            MEMBER   
  529  jupyterhub           oauthenticator            minrk            MEMBER   
  548  jupyterhub             team-compass      mael-le-gal            MEMBER   
  552  jupyterhub             team-compass         willingc            MEMBER   
  554  jupyterhub  the-littlest-jupyterhub      CyborgDroid              NONE   
  415  jupyterhub                binderhub         memeplex              NONE   
  456  jupyterhub               jupyterhub      brayanrodpr              NONE   
  492  jupyterhub              kubespawner           gsemet       CONTRIBUTOR   
  575  jupyterhub  the-littlest-jupyterhub  lachlancampbell              NONE   
  608  jupyterhub   zero-to-jupyterhub-k8s        lakshaykc              NONE   
  409  jupyterhub                binderhub          jhamman       CONTRIBUTOR   
  416  jupyterhub                binderhub            minrk            MEMBER   
  421  jupyterhub                binderhub        yuvipanda            MEMBER   
  462  jupyterhub               jupyterhub          eylenth              NONE   
  466  jupyterhub

Chart({
  data:             org                      repo             username  \
  718  jupyterlab                jupyterlab           jasongrout   
  668  jupyterlab                jupyterlab            blink1073   
  711  jupyterlab                jupyterlab           ian-r-rose   
  771  jupyterlab                jupyterlab              vidartf   
  761  jupyterlab                jupyterlab           telamonian   
  754  jupyterlab                jupyterlab       saulshanabrook   
  695  jupyterlab                jupyterlab            ellisonbg   
  652  jupyterlab                jupyterlab               afshin   
  630  jupyterlab                jupyterlab               BoPeng   
  794  jupyterlab            jupyterlab-git           jaipreet-s   
  762  jupyterlab                jupyterlab            tgeorgeux   
  781  jupyterlab  jupyterlab-data-explorer       saulshanabrook   
  659  jupyterlab                jupyterlab             aschlaep   
  690  jupyterlab                jupyterlab             domoritz   
  766  jupyterlab                jupyterlab              tslaton   
  716  jupyterlab                jupyterlab              jakevdp   
  719  jupyterlab                jupyterlab                jtpio   
  727  jupyterlab                jupyterlab           krassowski   
  649  jupyterlab                jupyterlab              Zsailer   
  693  jupyterlab                jupyterlab         duarteocarmo   
  773  jupyterlab                jupyterlab         vtrokhymenko   
  786  jupyterlab           jupyterlab-demo           ian-r-rose   
  787  jupyterlab           jupyterlab-demo           jasongrout   
  706  jupyterlab                jupyterlab              gnestor   
  712  jupyterlab                jupyterlab              ibdafna   
  739  jupyterlab                jupyterlab           michaelaye   
  759  jupyterlab                jupyterlab             stonebig   
  793  jupyterlab            jupyterlab-git           fcollonval   
  802  jupyterlab            jupyterlab-git       saulshanabrook   
  764  jupyterlab                jupyterlab            timkpaine   
  816  jupyterlab         jupyterlab_server            blink1073   
  636  jupyterlab                jupyterlab           IgorDouven   
  743  jupyterlab                jupyterlab           nickeubank   
  798  jupyterlab            jupyterlab-git  mathematicalmichael   
  666  jupyterlab                jupyterlab            bionicles   
  667  jupyterlab                jupyterlab              bjtho08   
  685  jupyterlab                jupyterlab         davidanthoff   
  694  jupyterlab                jupyterlab             echarles   
  701  jupyterlab                jupyterlab         flying-sheep   
  714  jupyterlab                jupyterlab               ivanov   
  715  jupyterlab                jupyterlab           jaipreet-s   
  717  jupyterlab                jupyterlab            jakirkham   
  777  jupyterlab                jupyterlab            yuvipanda   
  796  jupyterlab            jupyterlab-git           luispuerto   
  661  jupyterlab                jupyterlab            athornton   
  665  jupyterlab                jupyterlab              betatim   
  680  jupyterlab                jupyterlab           daejong123   
  684  jupyterlab                jupyterlab     david-waterworth   
  726  jupyterlab                jupyterlab              kevmk04   
  729  jupyterlab                jupyterlab           luispuerto   
  
      authorAssociation  author  
  718            MEMBER     418  
  668            MEMBER     220  
  711            MEMBER     123  
  771            MEMBER     121  
  761            MEMBER      50  
  754            MEMBER      47  
  695            MEMBER      44  
  652            MEMBER      41  
  630            MEMBER      21  
  794      COLLABORATOR      21  
  762            MEMBER      20  
  781            MEMBER      20  
  659            MEMBER      17  
  690            MEMBER      16  
  766

Chart({
  data:          org            repo             username authorAssociation  author
  867  nteract        hydrogen       wadethestealth            MEMBER     141
  846  nteract        hydrogen             aviatesk            MEMBER      66
  829  nteract       bookstore               mpacer            MEMBER      52
  880  nteract         nteract         captainsafia            MEMBER      40
  902  nteract       papermill                MSeal            MEMBER      37
  833  nteract       bookstore             willingc            MEMBER      35
  838  nteract        hydrogen           BenRussert            MEMBER      35
  912  nteract       papermill               rgbkrk            MEMBER      18
  893  nteract         nteract               rgbkrk            MEMBER      16
  832  nteract       bookstore                 todo              NONE      13
  896  nteract         nteract       wadethestealth            MEMBER      13
  915  nteract       scrapbook                MSeal            MEMBER      13
  872  nteract         nteract           BenRussert            MEMBER      12
  899  nteract      nteract.io                  now              NONE       9
  853  nteract        hydrogen           kiwi0fruit       CONTRIBUTOR       8
  874  nteract         nteract       CrystallineCat            MEMBER       8
  863  nteract        hydrogen             shr00mie              NONE       7
  907  nteract       papermill         duarteocarmo            MEMBER       7
  834  nteract        commuter         captainsafia            MEMBER       6
  836  nteract        commuter               taniki            MEMBER       6
  850  nteract        hydrogen             hoishing            MEMBER       6
  859  nteract        hydrogen            pepicello              NONE       6
  865  nteract        hydrogen             stevengj              NONE       6
  877  nteract         nteract         ShalokShalom              NONE       6
  911  nteract       papermill               mwouts            MEMBER       6
  828  nteract       bookstore                MSeal            MEMBER       5
  881  nteract         nteract         davidanthoff              NONE       5
  890  nteract         nteract      nicolaskruchten              NONE       5
  914  nteract       papermill             willingc            MEMBER       5
  922  nteract        semiotic               emeeks            MEMBER       5
  927  nteract            vdom             rmorshea            MEMBER       5
  858  nteract        hydrogen         optionsraghu              NONE       4
  864  nteract        hydrogen      sschuldenzucker              NONE       4
  897  nteract         nteract             willingc            MEMBER       4
  903  nteract       papermill          RoelBrouwer       CONTRIBUTOR       4
  904  nteract       papermill  avisrivastava254084              NONE       4
  905  nteract       papermill            casperdcl            MEMBER       4
  916  nteract       scrapbook           matthiasdv            MEMBER       4
  830  nteract       bookstore               rgbkrk            MEMBER       3
  851  nteract        hydrogen             isajacas              NONE       3
  852  nteract        hydrogen          jesusloplar              NONE       3
  856  nteract        hydrogen             n-riesco            MEMBER       3
  871  nteract  minimal-plotly               rgbkrk            MEMBER       3
  878  nteract         nteract          SpencerPark              NONE       3
  900  nteract       papermill               0anton              NONE       3
  925  nteract            vdom              gnestor            MEMBER       3
  835  nteract        commuter               guzmud              NONE       2
  839  nteract        hydrogen               Helveg              NONE       2
  840  nteract        hydrogen            Mikeymice              NONE       2
  842  nteract        hydrogen                Tbosa              NONE       2,
  encod

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [152]:
first_responders = created.copy().reset_index(drop=True)
for ii, issue in first_responders.iterrows():
    comments = issue.get('comments', [])['edges']
    if len(comments) == 0:
        continue
    i_comments = pd.DataFrame([ii['node'] for ii in comments]).sort_values('createdAt')
    first_comment = i_comments.iloc[0]
    first_responders.loc[ii, 'first_responder'] = first_comment['author']['login']
    first_responders.loc[ii, 'first_responder_association'] = first_comment['authorAssociation']

In [153]:
first_responders = (
    first_responders
    .groupby(['org', 'repo', 'first_responder', 'first_responder_association'])
    .count()
    .rename(columns={'author': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [154]:
n_plot = 50
for ii, (iorg, idata) in enumerate(first_responders.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='first_responder',
        y='count',
        color=alt.Color('first_responder_association', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org          repo first_responder first_responder_association  \
  0  binder-examples  jupyter-rise       choldgraf                      MEMBER   
  
     count  
  0      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:        org              repo first_responder first_responder_association  \
  3  ipython           ipython         Carreau                      MEMBER   
  1  ipython         ipykernel      ccordoba12                 CONTRIBUTOR   
  2  ipython           ipython    Benjamin-Lee                        NONE   
  4  ipython           ipython  LucianaMarques                 CONTRIBUTOR   
  5  ipython           ipython         dickmao                        NONE   
  6  ipython           ipython          dlukes                        NONE   
  7  ipython           ipython       gdhameeja                        NONE   
  8  ipython           ipython        jfhbrook                        NONE   
  9  ipython  ipython-in-depth      jasongrout                      MEMBER   
  
     count  
  3     11  
  1      1  
  2      1  
  4      1  
  5      1  
  6      1  
  7      1  
  8      1  
  9      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for ipython in the last 60 days',
  width: 1000
})

Chart({
  data:         org                repo first_responder first_responder_association  \
  66  jupyter         repo2docker         betatim                      MEMBER   
  18  jupyter        jupyter-book       choldgraf                      MEMBER   
  42  jupyter            nbgrader        jhamrick                      MEMBER   
  33  jupyter           nbconvert           MSeal                      MEMBER   
  16  jupyter  enterprise_gateway     kevin-bates                      MEMBER   
  13  jupyter       docker-stacks         parente                      MEMBER   
  19  jupyter        jupyter-book         emdupre                COLLABORATOR   
  34  jupyter           nbconvert        t-makaro                COLLABORATOR   
  38  jupyter              nbdime         vidartf                COLLABORATOR   
  46  jupyter            nbviewer        krinsman                 CONTRIBUTOR   
  30  jupyter      jupyter_server        rolweber                COLLABORATOR   
  32  jupyter               nb2kg     kevin-bates                      MEMBER   
  39  jupyter            nbgrader         Sefriol                        NONE   
  44  jupyter            nbgrader       perllaghu                        NONE   
  56  jupyter            notebook        jyu78749                        NONE   
  57  jupyter            notebook     kevin-bates                      MEMBER   
  64  jupyter           qtconsole      ccordoba12                COLLABORATOR   
  69  jupyter         repo2docker          manics                        NONE   
  10  jupyter       accessibility          fperez                      MEMBER   
  11  jupyter       docker-stacks       daviddoji                        NONE   
  12  jupyter       docker-stacks   ejgutierrez74                        NONE   
  14  jupyter       docker-stacks        richtong                        NONE   
  15  jupyter  enterprise_gateway          esevan                COLLABORATOR   
  17  jupyter             jupyter       choldgraf                      MEMBER   
  20  jupyter        jupyter-book           muzny                        NONE   
  21  jupyter        jupyter-book     thammegowda                 CONTRIBUTOR   
  22  jupyter      jupyter-sphinx    hugokerstens                 CONTRIBUTOR   
  23  jupyter      jupyter_client         dsblank                      MEMBER   
  24  jupyter      jupyter_client     kevin-bates                      MEMBER   
  25  jupyter      jupyter_client       takluyver                      MEMBER   
  26  jupyter      jupyter_client         vidartf                        NONE   
  27  jupyter      jupyter_server   SylvainCorlay                      MEMBER   
  28  jupyter      jupyter_server       blink1073                      MEMBER   
  29  jupyter      jupyter_server        lresende                      MEMBER   
  31  jupyter      kernel_gateway     kevin-bates                      MEMBER   
  35  jupyter           nbconvert          xiaohk                        NONE   
  36  jupyter              nbdime       bavaria95                        NONE   
  37  jupyter              nbdime        itamarst                        NONE   
  40  jupyter            nbgrader       Yenthe666                        NONE   
  41  jupyter            nbgrader         dsblank                      MEMBER   
  43  jupyter            nbgrader         ocefpaf                        NONE   
  45  jupyter            nbgrader     psychemedia                        NONE   
  47  jupyter            notebook    Elizaveta239                        NONE   
  48  jupyter            notebook          Iorent                        NONE   
  49  jupyter            notebook   WilliamTjiong                        NONE   
  50  jupyter            notebook         YuGlede                        NONE   
  51  jupyter            notebook    autodataming                        NONE   
  52  jupyter            notebook        coldJune                        NONE   
  53  jupyter    

Chart({
  data:                 org                repo first_responder  \
  78  jupyter-widgets          ipywidgets   SylvainCorlay   
  88  jupyter-widgets          ipywidgets         vidartf   
  92  jupyter-widgets           pythreejs         vidartf   
  82  jupyter-widgets          ipywidgets      jasongrout   
  74  jupyter-widgets          ipyleaflet     dwr-psandhu   
  75  jupyter-widgets          ipyleaflet     martinRenou   
  76  jupyter-widgets          ipyleaflet     paulathomas   
  77  jupyter-widgets          ipywidgets       JohnIssue   
  79  jupyter-widgets          ipywidgets          bybyte   
  80  jupyter-widgets          ipywidgets       choldgraf   
  81  jupyter-widgets          ipywidgets           hainm   
  83  jupyter-widgets          ipywidgets     jquacinella   
  84  jupyter-widgets          ipywidgets         kafonek   
  85  jupyter-widgets          ipywidgets           metya   
  86  jupyter-widgets          ipywidgets          piyueh   
  87  jupyter-widgets          ipywidgets      rskabelund   
  89  jupyter-widgets          ipywidgets     xilunaJumma   
  90  jupyter-widgets          ipywidgets     zxzhaixiang   
  91  jupyter-widgets  jupyterlab-sidecar      doutriaux1   
  93  jupyter-widgets            tutorial         vidartf   
  
     first_responder_association  count  
  78                      MEMBER      5  
  88                      MEMBER      4  
  92                      MEMBER      4  
  82                      MEMBER      3  
  74                        NONE      1  
  75                      MEMBER      1  
  76                        NONE      1  
  77                        NONE      1  
  79                        NONE      1  
  80                        NONE      1  
  81                        NONE      1  
  83                        NONE      1  
  84                        NONE      1  
  85                        NONE      1  
  86                        NONE      1  
  87                        NONE      1  
  89                        NONE      1  
  90                        NONE      1  
  91                        NONE      1  
  93                      MEMBER      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for jupyter-widgets in the last 60 days',
  width: 1000
})

Chart({
  data:             org                     repo     first_responder  \
  98   jupyterhub                binderhub             betatim   
  115  jupyterhub               jupyterhub             ryogesh   
  123  jupyterhub      mybinder.org-deploy             betatim   
  136  jupyterhub             team-compass             betatim   
  149  jupyterhub   zero-to-jupyterhub-k8s        consideRatio   
  127  jupyterhub              nbgitpuller           yuvipanda   
  139  jupyterhub  the-littlest-jupyterhub      GeorgianaElena   
  137  jupyterhub             team-compass           choldgraf   
  142  jupyterhub  the-littlest-jupyterhub     lachlancampbell   
  145  jupyterhub  the-littlest-jupyterhub           yuvipanda   
  148  jupyterhub              wrapspawner             rkdarst   
  96   jupyterhub             batchspawner             rkdarst   
  101  jupyterhub                binderhub           sgibson91   
  110  jupyterhub               jupyterhub            katsar0v   
  114  jupyterhub               jupyterhub          ryanlovett   
  119  jupyterhub              kubespawner        consideRatio   
  138  jupyterhub             team-compass               minrk   
  151  jupyterhub   zero-to-jupyterhub-k8s              manics   
  152  jupyterhub   zero-to-jupyterhub-k8s   sampathkethineedi   
  94   jupyterhub             batchspawner               Hoeze   
  95   jupyterhub             batchspawner            rcthomas   
  97   jupyterhub                binderhub               arnim   
  99   jupyterhub                binderhub        consideRatio   
  100  jupyterhub                binderhub              manics   
  102  jupyterhub            dockerspawner            Redhappy   
  103  jupyterhub            dockerspawner    ZelphirKaltstahl   
  104  jupyterhub            dockerspawner           timhughes   
  105  jupyterhub            dockerspawner         yakutovicha   
  106  jupyterhub     jupyter-server-proxy     juliusvonkohout   
  107  jupyterhub               jupyterhub      GeorgianaElena   
  108  jupyterhub               jupyterhub         UCSBGauchos   
  109  jupyterhub               jupyterhub           casperdcl   
  111  jupyterhub               jupyterhub              manics   
  112  jupyterhub               jupyterhub               minrk   
  113  jupyterhub               jupyterhub          rochaporto   
  116  jupyterhub               jupyterhub           vilhelmen   
  117  jupyterhub               jupyterhub            wqh17101   
  118  jupyterhub    kerberosauthenticator              jcrist   
  120  jupyterhub              kubespawner              gsemet   
  121  jupyterhub              kubespawner      julienchastang   
  122  jupyterhub        ldapauthenticator         dhirschfeld   
  124  jupyterhub      mybinder.org-deploy           choldgraf   
  125  jupyterhub              nbgitpuller      albertmichaelj   
  126  jupyterhub              nbgitpuller            mpilosov   
  128  jupyterhub           oauthenticator  hedgehog-millenium   
  129  jupyterhub           oauthenticator            inviscid   
  130  jupyterhub           oauthenticator               minrk   
  131  jupyterhub           oauthenticator           yuvipanda   
  132  jupyterhub           systemdspawner              Yensan   
  133  jupyterhub           systemdspawner            rstuckey   
  
      first_responder_association  count  
  98                       MEMBER      9  
  115                 CONTRIBUTOR      7  
  123                      MEMBER      7  
  136                      MEMBER      5  
  149                      MEMBER      5  
  127                      MEMBER      4  
  139                COLLABORATOR      4  
  137                      MEMBER      3  
  142                        NONE      3  
  145                      MEMBER      3  
  148                        NONE      3  
  96                  CONTRIBUTOR      2  
  101                      MEMBER      2  
  110          

Chart({
  data:             org                       repo  first_responder  \
  182  jupyterlab                 jupyterlab       jasongrout   
  198  jupyterlab                 jupyterlab          vidartf   
  169  jupyterlab                 jupyterlab        blink1073   
  186  jupyterlab                 jupyterlab       krassowski   
  202  jupyterlab   jupyterlab-data-explorer   saulshanabrook   
  179  jupyterlab                 jupyterlab       ian-r-rose   
  193  jupyterlab                 jupyterlab       telamonian   
  158  jupyterlab                 jupyterlab           BoPeng   
  159  jupyterlab                 jupyterlab       IgorDouven   
  165  jupyterlab                 jupyterlab           afshin   
  174  jupyterlab                 jupyterlab         echarles   
  175  jupyterlab                 jupyterlab        ellisonbg   
  183  jupyterlab                 jupyterlab            jtpio   
  196  jupyterlab                 jupyterlab          tslaton   
  155  jupyterlab  extension-cookiecutter-ts       jasongrout   
  156  jupyterlab          jupyter-renderers        jonmmease   
  157  jupyterlab                 jupyterlab      AlokTakshak   
  160  jupyterlab                 jupyterlab        Koushik95   
  161  jupyterlab                 jupyterlab         SamKacer   
  162  jupyterlab                 jupyterlab         SgtPooki   
  163  jupyterlab                 jupyterlab  SultanOrazbayev   
  164  jupyterlab                 jupyterlab            Zlass   
  166  jupyterlab                 jupyterlab       alesgenova   
  167  jupyterlab                 jupyterlab     arogozhnikov   
  168  jupyterlab                 jupyterlab         aschlaep   
  170  jupyterlab                 jupyterlab         chriotte   
  171  jupyterlab                 jupyterlab      claraabello   
  172  jupyterlab                 jupyterlab       daejong123   
  173  jupyterlab                 jupyterlab       doutriaux1   
  176  jupyterlab                 jupyterlab     felixdollack   
  177  jupyterlab                 jupyterlab     flying-sheep   
  178  jupyterlab                 jupyterlab         groverj3   
  180  jupyterlab                 jupyterlab          ibdafna   
  181  jupyterlab                 jupyterlab       jaipreet-s   
  184  jupyterlab                 jupyterlab        kanekalla   
  185  jupyterlab                 jupyterlab        kapilkd13   
  187  jupyterlab                 jupyterlab    markellekelly   
  188  jupyterlab                 jupyterlab        marwahaha   
  189  jupyterlab                 jupyterlab     michael-burt   
  190  jupyterlab                 jupyterlab   mouradmourafiq   
  191  jupyterlab                 jupyterlab   saulshanabrook   
  192  jupyterlab                 jupyterlab         stonebig   
  194  jupyterlab                 jupyterlab        tgeorgeux   
  195  jupyterlab                 jupyterlab        timkpaine   
  197  jupyterlab                 jupyterlab            van-8   
  199  jupyterlab        jupyterlab-celltags            e01n0   
  200  jupyterlab      jupyterlab-commenting          gogasca   
  201  jupyterlab      jupyterlab-commenting           hoo761   
  203  jupyterlab             jupyterlab-git      DiegoPiloni   
  204  jupyterlab             jupyterlab-git       fcollonval   
  
      first_responder_association  count  
  182                      MEMBER     25  
  198                      MEMBER      9  
  169                      MEMBER      8  
  186                      MEMBER      4  
  202                      MEMBER      4  
  179                      MEMBER      3  
  193                      MEMBER      3  
  158                      MEMBER      2  
  159                        NONE      2  
  165                      MEMBER      2  
  174                      MEMBER      2  
  175                      MEMBER      2  
  183                      MEMBER      2  
  196                 CONTRIBUTOR      2  
  155                  

Chart({
  data:          org            repo  first_responder first_responder_association  \
  228  nteract       papermill            MSeal                      MEMBER   
  222  nteract         nteract     captainsafia                      MEMBER   
  218  nteract        hydrogen   wadethestealth                      MEMBER   
  212  nteract       bookstore           mpacer                      MEMBER   
  225  nteract         nteract   wadethestealth                      MEMBER   
  230  nteract       scrapbook            MSeal                      MEMBER   
  214  nteract       bookstore         willingc                      MEMBER   
  216  nteract        hydrogen         aviatesk                      MEMBER   
  211  nteract       bookstore            MSeal                      MEMBER   
  213  nteract       bookstore           rgbkrk                      MEMBER   
  215  nteract              dx       dylanross3                      MEMBER   
  217  nteract        hydrogen         n-riesco                      MEMBER   
  219  nteract  minimal-plotly  nicolaskruchten                        NONE   
  220  nteract         nteract       BenRussert                      MEMBER   
  221  nteract         nteract          bnbarak                      MEMBER   
  223  nteract         nteract             drbh                        NONE   
  224  nteract         nteract           rgbkrk                      MEMBER   
  226  nteract         nteract         willingc                      MEMBER   
  227  nteract         nteract       yogevyuval                        NONE   
  229  nteract       papermill         willingc                      MEMBER   
  231  nteract        semiotic      micahstubbs                 CONTRIBUTOR   
  232  nteract            vdom         rmorshea                      MEMBER   
  
       count  
  228     10  
  222      5  
  218      4  
  212      3  
  225      3  
  230      3  
  214      2  
  216      2  
  211      1  
  213      1  
  215      1  
  217      1  
  219      1  
  220      1  
  221      1  
  223      1  
  224      1  
  226      1  
  227      1  
  229      1  
  231      1  
  232      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for nteract in the last 60 days',
  width: 1000
})

## List of all contributors per organization

In [155]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_closed = closed.query("org == @org")
    commentors_closed = commentors.query("org == @org")
    unique_participants = np.unique(np.hstack([commentors_closed['username'].unique(), authors_closed['username'].unique()]))
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are all participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

Here are all participants across issues and pull requests in each org in the last 60 days

In [156]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>